In [4]:
# visualize feature maps output from each block in the vgg model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model
from matplotlib import pyplot
from numpy import expand_dims
import numpy as np
from itertools import chain
from keras import backend as K
from keras.layers.convolutional import MaxPooling3D
from vvvv import VVVVClient
import keras
import time
import socket

In [10]:
def mainCycle(model, ixs, path, cycle):
    
    outputs = [model.layers[i].output for i in ixs]
    model = Model(inputs = model.inputs, outputs = outputs)
    
    # load the image with the required shape
    img = load_img(path, target_size=(224, 224))

    # convert the image to an array
    img = img_to_array(img)

    # expand dimensions so that it represents a single 'sample'
    img = expand_dims(img, axis=0)
    # prepare the image (e.g. scale pixel values for the vgg)
    img = preprocess_input(img)

    # get feature map for first hidden layer
    feature_maps = model.predict(img)


    toMax = []

    for i in range(len(feature_maps)):
        toMax.append(feature_maps[i][0].mean(2).flatten().tolist())

    #print(toMax[4])

    toMaxArray = []
    #print(toMaxArray)
    print(cycle)
    
    func = [3, 2, 1, 1, 1]

    for i in range (len(toMax)):
        n = feature_maps[i][0].shape[0]
        pool = keras.layers.MaxPooling2D(pool_size=(func[i], func[i]))
        b = K.constant(feature_maps[i][0].mean(2).reshape(-1, n, n, 1))
        out = K.eval(pool(b))
        out = (out - out.min()) / (out.max() - out.min())
        #print(out.shape)
        toMaxArray += [out.flatten().tolist()]
    return toMaxArray

def setDelayTime(ip, port):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    buffer_size = 8192
    s.bind((ip, port))
    #print(client, port)
    s.listen(1)
    (client, (ip,port)) = s.accept()
    s.settimeout(1)

    while True:
        data = client.recv(buffer_size)
        delayTime = 0
        dT = 0

        if len(data) == 0:
#                 print(dT)
            s.close()
            break
        else:
            # got a message do something :)
            delayTime = data.decode()
            dT = float(delayTime.replace('\n', '').replace(';', ''))
            print(dT)
            vc = VVVVClient('127.0.0.1', int('16161'))
            vc.send_msg('/fb', str(1))
            time.sleep(.2)
            vc.send_msg('/fb', str(0))
        s.close()
        return dT

def vcClient(ip, port, toMaxArray, dT = 0.5):
    vc = VVVVClient('127.0.0.1', int('12000'))
    for i in range(len(toMaxArray)):
        vc.send_msg('/conv{}'.format(i), toMaxArray[i])
        vc.send_msg('/trig', str(1))
        time.sleep(.2)
        vc.send_msg('/trig', str(0))
    time.sleep(dT)
#     print('sucks')

In [11]:
cycle = 0
dT = 0.5

# load the model
model = VGG16()

# redefine model to output right after the first hidden layer
ixs = [2, 5, 9, 13, 17]


toMaxArray = mainCycle(model, ixs, r"C:\Users\adchu\Desktop\0.png", cycle)
vcClient('127.0.0.1', '12000', toMaxArray, dT)

# mainCycle(model, ixs, r"C:\Users\adchu\Desktop\1.png", cycle)
# vcClient('127.0.0.1', '12000', toMaxArray, dT)

while True:
    cycle += 1
    toMaxArray = mainCycle(model, ixs, r"C:\Users\adchu\Desktop\1.png", cycle)
#     dT = setDelayTime('127.0.0.1', 12002)
    vcClient('127.0.0.1', '12000', toMaxArray, dT)

0
1
2
3
4
5
6
7
8
9
10
11
12
13


KeyboardInterrupt: 